In [131]:
import requests
import json
import os
from google.transit import gtfs_realtime_pb2
import hmac
from hashlib import sha1
import pandas as pd
import numpy as np
import psycopg2
import datetime

def parse_gtfs_object(entity):
    if 'ListFields' not in dir(entity):
        return entity
    entity_dict = {}
    for field in entity.ListFields():
        field_name = field[0].name
        if field[0].label == field[0].LABEL_REPEATED:
            field_value = [parse_gtfs_object(item) for item in field[1]]
        else:
            field_value = parse_gtfs_object(field[1])
        entity_dict[field_name] = field_value
    return entity_dict

def parse_gtfs_realtime_feed(feed):
    return [parse_gtfs_object(entity) for entity in feed.entity]

def parse_gtfs_r_binary(feed_data):
    """
    Example usage:
    response = requests.get(URL)
    feed_data = response.content
    feed_json = parse_gtfs_r_binary(feed_data)
    """
    feed = gtfs_realtime_pb2.FeedMessage()
    feed.ParseFromString(feed_data)
    return parse_gtfs_realtime_feed(feed)


def get_ptv_api_url(
        endpoint : str,
        dev_id : str | int, 
        api_key : str | int,
    ):
    """
    Returns the URL to use PTV TimeTable API.

    Generates a signature from dev id (user id), API key, and endpoint.

    See the following for more information:
    - Home page: https://www.ptv.vic.gov.au/footer/data-and-reporting/datasets/ptv-timetable-api/
    - Swagger UI: https://timetableapi.ptv.vic.gov.au/swagger/ui/index
    - Swagger Docs JSON: https://timetableapi.ptv.vic.gov.au/swagger/docs/v3 (You can use this to find the endpoints you want to use.)
    """
    assert endpoint.startswith('/'), f'Endpoint must start with /, got {endpoint}'
    raw = f'{endpoint}{'&' if '?' in endpoint else '?'}devid={dev_id}'
    hashed = hmac.new(api_key.encode('utf-8'), raw.encode('utf-8'), sha1)  # Encode the raw string to bytes
    signature = hashed.hexdigest()
    return f'https://timetableapi.ptv.vic.gov.au{raw}&signature={signature}'


class PTVAPIClient:
    def __init__(self, dev_id : str | int, api_key : str | int):
        self.dev_id = dev_id
        self.api_key = api_key
        self.session = requests.Session()

    def get_data(self, endpoint : str, need_auth : bool = True):
        """
        Returns the data from the URL.
        """
        if need_auth:
            url = get_ptv_api_url(endpoint, self.dev_id, self.api_key)
        else:
            url = f'https://timetableapi.ptv.vic.gov.au{endpoint}'
        response = self.session.get(url)
        response.raise_for_status()
        return response.json()
    

class PTVAPI3(PTVAPIClient):
    def __init__(self, dev_id : str | int, api_key : str | int):
        super().__init__(dev_id, api_key)
        
    def get_docs(self) -> dict:
        return self.get_data('/swagger/docs/v3', need_auth=False)
    
    def get_routes(self) -> dict:
        return self.get_data('/v3/routes')['routes']
    
    def get_route_types(self) -> dict:
        return self.get_data('/v3/route_types')['route_types']
    
    def get_disruptions(self) -> dict:
        return self.get_data('/v3/disruptions')['disruptions']
    
    def get_disruption_modes(self) -> dict:
        return self.get_data('/v3/disruptions/modes')['disruption_modes']
    
    def get_outlets(self) -> dict:
        return self.get_data('/v3/outlets')['outlets']

In [132]:
ENV = json.load(open('../local-env.json'))
URLS = {
    'tram-servicealert': 'https://data-exchange-api.vicroads.vic.gov.au/opendata/gtfsr/v1/tram/servicealert',
    'tram-tripupdates': 'https://data-exchange-api.vicroads.vic.gov.au/opendata/gtfsr/v1/tram/tripupdates',
    'tram-vehicleposition': 'https://data-exchange-api.vicroads.vic.gov.au/opendata/gtfsr/v1/tram/vehicleposition',
    'bus-tripupdates': 'https://data-exchange-api.vicroads.vic.gov.au/opendata/v1/gtfsr/metrobus-tripupdates',
    'train-servicealerts': 'https://data-exchange-api.vicroads.vic.gov.au/opendata/v1/gtfsr/metrotrain-servicealerts',
    'train-tripupdates': 'https://data-exchange-api.vicroads.vic.gov.au/opendata/v1/gtfsr/metrotrain-tripupdates',
    'train-vehicleposition-updates': 'https://data-exchange-api.vicroads.vic.gov.au/opendata/v1/gtfsr/metrotrain-vehicleposition-updates',
}
CLIENT = PTVAPI3(ENV['PTV_TIMETABLE_DEV_ID'], ENV['PTV_TIMETABLE_API_KEY'])
SESSION = requests.Session()
FEED = gtfs_realtime_pb2.FeedMessage()
CONN = psycopg2.connect(
    dbname='gtfs',
    user='postgres',
    password='postgres',
    host='localhost',
    port='5432',
)
CURSOR = CONN.cursor()

In [74]:
DOCS = CLIENT.get_docs()['paths']
ROUTES = CLIENT.get_routes()

In [76]:
[route for route in ROUTES if route['route_number'] == '733']

[{'route_service_status': {'description': 'Good Service',
   'timestamp': '2024-03-27T06:02:24.1974949+00:00'},
  'route_type': 2,
  'route_id': 13271,
  'route_name': 'Oakleigh - Box Hill via Clayton & Monash University & Mt Waverley',
  'route_number': '733',
  'route_gtfs_id': '4-733',
  'geopath': []}]

In [78]:
CLIENT.get_data('/v3/directions/route/13271')

{'directions': [{'route_direction_description': 'Departs from the terminus at Oakleigh Railway Station (south side) and travels via Johnson St, Haughton Rd, Bank St, Golf Links Ave, Golf Rd, Cameron Ave, Centre Rd, Clayton Rd, Haughton Rd, Link Rd and Carinish Rd at Clayton Railway Station. From Clayton Railway Station bus travels via Clayton Rd, North Rd and Wellington Rd to Monash University (Clayton Campus); then via Wellington Rd, Princes Hwy, Clayton Rd, Stephensons Rd and Alexander Pl to Mount Waverley Bus Terminal; then via Stephensons Rd, Middleborough Rd, Albion Rd, Barcelona St and Rutland Rd to the terminus at Box Hill Railway Station.',
   'direction_id': 30,
   'direction_name': 'Box Hill',
   'route_id': 13271,
   'route_type': 2},
  {'route_direction_description': 'Departs from the Box Hill terminus via Carrington Rd, Ellingworth Pde, William St, Rutland Rd, Barcelona St, Albion Rd, Middleborough Rd, Stephensons Rd, Clayton Rd, Princes Hwy and Wellington Rd to Monash Uni

In [79]:
stops = CLIENT.get_data('/v3/stops/route/13271/route_type/2?direction_id=182')
stops = stops['stops']

In [114]:
DOCS['/v3/departures/route_type/{route_type}/stop/{stop_id}']['get']['parameters']

[{'name': 'route_type',
  'in': 'path',
  'description': 'Number identifying transport mode; values returned via RouteTypes API',
  'required': True,
  'type': 'integer',
  'format': 'int32',
  'enum': [0, 1, 2, 3, 4]},
 {'name': 'stop_id',
  'in': 'path',
  'description': 'Identifier of stop; values returned by Stops API',
  'required': True,
  'type': 'integer',
  'format': 'int32'},
 {'name': 'platform_numbers',
  'in': 'query',
  'description': 'Filter by platform number at stop',
  'required': False,
  'type': 'array',
  'items': {'type': 'integer', 'format': 'int32'},
  'collectionFormat': 'multi'},
 {'name': 'direction_id',
  'in': 'query',
  'description': 'Filter by identifier of direction of travel; values returned by Directions API - /v3/directions/route/{route_id}',
  'required': False,
  'type': 'integer',
  'format': 'int32'},
 {'name': 'gtfs',
  'in': 'query',
  'description': 'Indicates that stop_id parameter will accept "GTFS stop_id" data',
  'required': False,
  'typ

In [104]:
pd.DataFrame(DOCS['/v3/departures/route_type/{route_type}/stop/{stop_id}']['get']['parameters'])

,name,in,description,required,type,format,enum,items,collectionFormat
0,route_type,path,Number identifying transport mode; values retu...,True,integer,int32,"[0, 1, 2, 3, 4]",NaN,NaN
1,stop_id,path,Identifier of stop; values returned by Stops API,True,integer,int32,NaN,NaN,NaN
2,platform_numbers,query,Filter by platform number at stop,False,array,NaN,NaN,"{'type': 'integer', 'format': 'int32'}",multi
3,direction_id,query,Filter by identifier of direction of travel; v...,False,integer,int32,NaN,NaN,NaN
4,gtfs,query,"Indicates that stop_id parameter will accept ""...",False,boolean,NaN,NaN,NaN,NaN
5,date_utc,query,Filter by the date and time of the request (IS...,False,string,date-time,NaN,NaN,NaN
6,max_results,query,Maximum number of results returned,False,integer,int32,NaN,NaN,NaN
7,include_cancelled,query,Indicates if cancelled services (if they exist...,False,boolean,NaN,NaN,NaN,NaN
8,look_backwards,query,Indicates if filtering runs (and their departu...,False,boolean,NaN,NaN,NaN,NaN
9,expand,query,List of objects to be returned in full (i.e. e...,False,array,NaN,NaN,"{'type': 'string', 'enum': ['All', 'Stop', 'Ro...",multi


In [86]:
[stop for stop in stops if 'Woodside' in stop['stop_name']]

[{'disruption_ids': [],
  'stop_suburb': 'Clayton',
  'route_type': 2,
  'stop_latitude': -37.90909,
  'stop_longitude': 145.122787,
  'stop_sequence': 45,
  'stop_ticket': {'ticket_type': '',
   'zone': 'Zone 2',
   'is_free_fare_zone': False,
   'ticket_machine': False,
   'ticket_checks': False,
   'vline_reservation': False,
   'ticket_zones': [2]},
  'stop_id': 22752,
  'stop_name': 'Woodside Ave/Clayton Rd',
  'stop_landmark': ''}]

In [87]:
CLIENT.get_data('/v3/stops/22752/route_type/2')

{'stop': {'point_id': 6423,
  'operating_hours': 'N',
  'mode_id': 1,
  'station_details_id': 0,
  'flexible_stop_opening_hours': '',
  'stop_contact': None,
  'stop_ticket': None,
  'disruption_ids': [],
  'station_type': None,
  'station_description': None,
  'route_type': 2,
  'stop_location': None,
  'stop_amenities': None,
  'stop_accessibility': None,
  'stop_staffing': None,
  'routes': [{'route_type': 2,
    'route_id': 13271,
    'route_name': 'Oakleigh - Box Hill via Clayton & Monash University & Mt Waverley',
    'route_number': '733',
    'route_gtfs_id': '4-733',
    'geopath': []}],
  'stop_id': 22752,
  'stop_name': 'Woodside Ave/Clayton Rd ',
  'stop_landmark': ''},
 'disruptions': {},
 'status': {'version': '3.0', 'health': 1}}

In [109]:
runs = CLIENT.get_data('/v3/runs/route/13271')
runs = runs['runs']

In [120]:
df = pd.DataFrame(runs)
df[df['run_ref'].str.contains('101')]

,run_id,run_ref,route_id,route_type,final_stop_id,destination_name,status,direction_id,run_sequence,express_stop_count,vehicle_position,vehicle_descriptor,geopath
100,-1,20-733--1-MF2-101,13271,2,13989,Oakleigh Station/Johnson St,scheduled,182,100,0,None,None,[]


In [ ]:
DOCS['/v3/runs/{run_ref}/route_type/{route_type}']

In [125]:
CLIENT.get_data('/v3/runs/20-733--1-MF2-101/route_type/2?direction_id=182')

{'run': {'run_id': -1,
  'run_ref': '20-733--1-MF2-101',
  'route_id': 13271,
  'route_type': 2,
  'final_stop_id': 13989,
  'destination_name': 'Oakleigh Station/Johnson St ',
  'status': 'scheduled',
  'direction_id': 182,
  'run_sequence': 0,
  'express_stop_count': 0,
  'vehicle_position': None,
  'vehicle_descriptor': None,
  'geopath': []},
 'status': {'version': '3.0', 'health': 1}}

In [146]:
CURSOR.execute("SELECT * FROM stop_times_4 WHERE stop_id = '8196' OR stop_id = '6423';")
results = CURSOR.fetchall()
columns = [desc[0] for desc in CURSOR.description]
results_df = pd.DataFrame(results, columns=columns)

In [148]:
results_df['stop_sequence'].unique()
# 6423	Woodside Ave/Clayton Rd (Clayton)	-37.9090919728207	145.122785181621
# 8196	20-733--1-MF4-101
results_df[results_df['trip_id'] == '20-733--1-MF4-101']

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
349,20-733--1-MF4-101,18:52:00,18:52:00,6423,45,,0,0,11602.70


In [205]:
departures = CLIENT.get_data('/v3/departures/route_type/2/stop/22752?expand=All&direction_id=182')['departures']
departure = [departure for departure in departures if (departure['estimated_departure_utc'] is not None) and datetime.datetime.fromisoformat(departure['estimated_departure_utc']).astimezone().timestamp() >= datetime.datetime.now().timestamp()][0]
scheduled_departure = datetime.datetime.fromisoformat(departure['scheduled_departure_utc']).astimezone()
estimated_departure = datetime.datetime.fromisoformat(departure['estimated_departure_utc']).astimezone()
scheduled_departure, estimated_departure, departure['run_ref']

(datetime.datetime(2024, 3, 27, 18, 52, tzinfo=datetime.timezone(datetime.timedelta(seconds=39600), 'AUS Eastern Daylight Time')),
 datetime.datetime(2024, 3, 27, 19, 4, 10, tzinfo=datetime.timezone(datetime.timedelta(seconds=39600), 'AUS Eastern Daylight Time')),
 '20-733--1-MF2-101')

In [204]:
for gtfs_api_key in [ENV['GTFSR_PRIMARY_KEY'], ENV['GTFSR_SECONDARY_KEY']]:
    response = SESSION.get(URLS['bus-tripupdates'], headers={
        # Request headers
        'Cache-Control': 'no-cache',
        'Ocp-Apim-Subscription-Key': gtfs_api_key,
    })
    if response.status_code == 200:
        break
FEED.ParseFromString(response.content)
entities = [entity for entity in FEED.entity if entity.trip_update.trip.trip_id.split('-')[1] == '733']

# [stop for stop in entities[0].trip_update.stop_time_update if stop.stop_sequence == 45]
# Get by stop-sequence = 45
stop = [stop for stop in entities[0].trip_update.stop_time_update if stop.stop_sequence == 45][0]
# Convert Unix timestamp to datetime
datetime.datetime.fromtimestamp(stop.arrival.time), datetime.datetime.fromtimestamp(stop.departure.time), entities[0].trip_update.trip.trip_id, stop

(datetime.datetime(2024, 3, 27, 19, 4, 10),
 datetime.datetime(2024, 3, 27, 19, 4, 10),
 '20-733--1-MF4-101',
 stop_sequence: 45
 arrival {
   time: 1711526650
 }
 departure {
   time: 1711526650
 })

In [124]:
[entity.trip_update.trip.trip_id for entity in entities]

['20-733--1-MF4-101',
 '20-733--1-MF4-159',
 '20-733--1-MF4-161',
 '20-733--1-MF4-261',
 '20-733--1-MF4-263',
 '20-733--1-MF4-265',
 '20-733--1-MF4-267',
 '20-733--1-MF4-329',
 '20-733--1-MF4-331',
 '20-733--1-MF4-97',
 '20-733--1-MF4-99']

In [30]:
parse_gtfs_object(entities[0])['trip_update']['stop_time_update']

[{'stop_sequence': 11,
  'arrival': {'time': 1711516746},
  'departure': {'time': 1711516746}},
 {'stop_sequence': 12,
  'arrival': {'time': 1711516764},
  'departure': {'time': 1711516764}},
 {'stop_sequence': 13,
  'arrival': {'time': 1711516808},
  'departure': {'time': 1711516808}},
 {'stop_sequence': 14,
  'arrival': {'time': 1711516845},
  'departure': {'time': 1711516845}},
 {'stop_sequence': 15,
  'arrival': {'time': 1711516893},
  'departure': {'time': 1711516893}},
 {'stop_sequence': 16,
  'arrival': {'time': 1711516929},
  'departure': {'time': 1711516929}},
 {'stop_sequence': 17,
  'arrival': {'time': 1711516956},
  'departure': {'time': 1711516956}},
 {'stop_sequence': 18,
  'arrival': {'time': 1711516978},
  'departure': {'time': 1711516978}},
 {'stop_sequence': 19,
  'arrival': {'time': 1711517015},
  'departure': {'time': 1711517015}},
 {'stop_sequence': 20,
  'arrival': {'time': 1711517056},
  'departure': {'time': 1711516461}},
 {'stop_sequence': 21,
  'arrival': {'ti

In [117]:
# Get information of bus route 733 
d = set()
for entity in FEED.entity:
    if entity.HasField('trip_update'):
        trip_id = entity.trip_update.trip.trip_id
        trip_idx = trip_id.split('-')
        if trip_idx[1] == '733':
            print(trip_id)
d

20-733--1-MF4-101
20-733--1-MF4-159
20-733--1-MF4-161
20-733--1-MF4-261
20-733--1-MF4-263
20-733--1-MF4-265
20-733--1-MF4-267
20-733--1-MF4-329
20-733--1-MF4-331
20-733--1-MF4-97
20-733--1-MF4-99


set()